In [1]:
from transformers import pipeline
import pandas as pd
import torch
from tqdm import tqdm

2023-04-19 08:22:04.388089: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 08:22:07.562623: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-19 08:22:07.562709: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-19 08:22:07.562717: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [3]:
print("Reading data")
data = pd.read_csv("data/Russia_invade.csv")

Reading data


In [2]:
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

In [8]:
candidate_labels = ['USA started the war between Russia and Ukraine',
                    'POTUS started the war between Russia and Ukraine',
                    'Joe Biden started the war between Russia and Ukraine',
                    'CIA started the war between Russia and Ukraine',
                    'USA influenced the war between Russia and Ukraine',
                    'POTUS influenced the war between Russia and Ukraine',
                    'Joe Biden influenced the war between Russia and Ukraine',
                    'CIA influenced the war between Russia and Ukraine']

In [6]:
print("Creating dataloader")
tweets = data["renderedContent"].tolist()[:10]

Creating dataloader


In [20]:
print("Starting analysis")
tweets_blaming_america = {}
for tweet in tqdm(tweets):
    result = classifier(tweet, candidate_labels, multi_label=True)
    if any(val > 0.75 for val in result['scores']):
        tweets_blaming_america[tweet] = {l: s for l, s in zip(result['labels'], result['scores'])}

Starting analysis


100%|██████████| 10/10 [00:19<00:00,  1.97s/it]


In [21]:
print(tweets_blaming_america)

{'JOE BIDEN SAYS HOW DO WE GET TO A PLACE WHERE PUTIN DECIDES TO INVADE RUSSIA 🤣🤣🤣😭😭😂😂\nPEOPLE VOTED FOR THIS GUY 🤣🤣🤣 https://t.co/FOHozS7vYI': {'Joe Biden influenced the war': 0.958122730255127, 'Joe Biden started the war': 0.06567798554897308, 'USA influenced the war': 0.061619728803634644, 'CIA influenced the war': 0.04029559716582298, 'POTUS influenced the war': 0.019074110314249992, 'CIA started the war': 0.0029467542190104723, 'POTUS started the war': 0.0004867762327194214, 'USA started the war': 0.0004355523851700127}, 'America forced Putin to invade in order to bog down Russia. Because Americans are evil and are sitting in their White House cackling evilly, devising wicked plans.\n\n10/10 genius logic Glenn https://t.co/zHHGG5amqX': {'USA influenced the war': 0.9927944540977478, 'USA started the war': 0.9240317344665527, 'POTUS influenced the war': 0.9187301397323608, 'POTUS started the war': 0.42189261317253113, 'CIA influenced the war': 0.049384839832782745, 'CIA started the 

In [22]:
for tweet, result in tweets_blaming_america.items():
    print(tweet.replace("\n", " "))
    for label, score in result.items():
        print(f"\t{label}: {score}")

JOE BIDEN SAYS HOW DO WE GET TO A PLACE WHERE PUTIN DECIDES TO INVADE RUSSIA 🤣🤣🤣😭😭😂😂 PEOPLE VOTED FOR THIS GUY 🤣🤣🤣 https://t.co/FOHozS7vYI
	Joe Biden influenced the war: 0.958122730255127
	Joe Biden started the war: 0.06567798554897308
	USA influenced the war: 0.061619728803634644
	CIA influenced the war: 0.04029559716582298
	POTUS influenced the war: 0.019074110314249992
	CIA started the war: 0.0029467542190104723
	POTUS started the war: 0.0004867762327194214
	USA started the war: 0.0004355523851700127
America forced Putin to invade in order to bog down Russia. Because Americans are evil and are sitting in their White House cackling evilly, devising wicked plans.  10/10 genius logic Glenn https://t.co/zHHGG5amqX
	USA influenced the war: 0.9927944540977478
	USA started the war: 0.9240317344665527
	POTUS influenced the war: 0.9187301397323608
	POTUS started the war: 0.42189261317253113
	CIA influenced the war: 0.049384839832782745
	CIA started the war: 0.039775583893060684
	Joe Biden in

In [9]:
tests = ['Lying propagandist media shouting invasion over Ukraine, yet remain silent on the ACTUAL INVASION of America at our southern border']
for tweet in tests:
    result = classifier(tweet, candidate_labels, multi_label=True)
    print(tweet)
    for label, score in zip(result['labels'], result['scores']):
        print(f"\t{label}: {score}")

Lying propagandist media shouting invasion over Ukraine, yet remain silent on the ACTUAL INVASION of America at our southern border
	USA influenced the war between Russia and Ukraine: 0.0728783905506134
	CIA influenced the war between Russia and Ukraine: 0.00525757297873497
	POTUS influenced the war between Russia and Ukraine: 0.0025946798268705606
	USA started the war between Russia and Ukraine: 0.0016453873831778765
	Joe Biden influenced the war between Russia and Ukraine: 0.0016119081992655993
	CIA started the war between Russia and Ukraine: 0.00143335887696594
	POTUS started the war between Russia and Ukraine: 0.0006974498392082751
	Joe Biden started the war between Russia and Ukraine: 0.0006885910406708717
